In [6]:
import pandas as pd

df_sjr = pd.read_csv("scimagojr 2022.csv", sep=";").drop("Rank", axis=1)
df_sjr 

,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2022),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
0,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","86,091",Q1,198,44,118,4268,30318,85,"299,99","97,00",United States,Northern America,Wiley-Blackwell,1950-2022,Hematology (Q1); Oncology (Q1),Medicine
1,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","36,730",Q1,292,36,122,2398,2141,122,"14,83","66,61",United Kingdom,Western Europe,Oxford University Press,1886-2022,Economics and Econometrics (Q1),"Economics, Econometrics and Finance"
2,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","34,201",Q1,485,121,328,10804,13331,156,"35,47","89,29",United Kingdom,Western Europe,Nature Publishing Group,2000-2022,Cell Biology (Q1); Molecular Biology (Q1),"Biochemistry, Genetics and Molecular Biology"
3,18434,Cell,journal,"00928674, 10974172","26,494",Q1,856,420,1637,27609,67791,1440,"43,80","65,74",United States,Northern America,Cell Press,1974-2022,"Biochemistry, Genetics and Molecular Biology (...","Biochemistry, Genetics and Molecular Biology"
4,15847,New England Journal of Medicine,journal,"00284793, 15334406","26,015",Q1,1130,1410,4561,14396,133956,1854,"33,93","10,21",United States,Northern America,Massachussetts Medical Society,1945-2022,Medicine (miscellaneous) (Q1),Medicine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27950,21101091736,Worldwide Waste,journal,23997117,NaN,-,1,5,0,320,0,0,"0,00","64,00",United Kingdom,Western Europe,Web Portal Ubiquity Press,2022,Waste Management and Disposal,Environmental Science
27951,17700156005,WSEAS Transactions on Heat and Mass Transfer (...,journal,17905044,NaN,-,10,24,0,593,0,0,"0,00","24,71",Greece,Western Europe,World Scientific and Engineering Academy and S...,"2009-2015, 2022",Condensed Matter Physics; Energy (miscellaneou...,Energy; Engineering; Physics and Astronomy
27952,21100829466,Yearbook of Comparative Literature,journal,"19255772, 19255764",NaN,-,4,9,0,412,0,0,"0,00","45,78",Canada,Northern America,University of Toronto Press,"2009-2011, 2013-2017",Cultural Studies; History; Literature and Lite...,Arts and Humanities; Social Sciences
27953,21100212328,Zbornik za Umetnostno Zgodovino,journal,"0351224X, 15803767",NaN,-,3,1,0,0,0,0,"0,00","0,00",Slovenia,Eastern Europe,Slovene Art History Society,2011-2018,History; Visual Arts and Performing Arts,Arts and Humanities
